In [ ]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
print(OpenAI.api_key)

In [ ]:
instructions = [
    "As focus app 'FLORA' intelligent customer service chatbot, my goal is to assist you effectively.",
    "To ensure accuracy, I'll confirm your input and distill it into key points. Then, I'll list these points using bullet points and confirm with you if they're correct.",
    "If you have alternative descriptions for any key point, I'll update the description accordingly.",
    "We'll repeat this process until you're satisfied.",
]

instruction = ""
for i in instructions:
    instruction += i

print(instruction)

In [ ]:
assistant = client.beta.assistants.create(
    name = "Intelligent customer service chatbot",
    instructions = instruction,
    model = "gpt-3.5-turbo-1106"
)
thread = client.beta.threads.create()

# Start running from here

In [ ]:
user_input = ""

In [ ]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = user_input
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

In [ ]:
while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
print(run.status)

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

In [ ]:
for i in messages.data[-10:]:
    print(i.content[0].text.value)
    print("==========")

# 保留重點

In [ ]:
i = "I understand. So, your main concerns are:1. Trees being killed unjustly and tasks not being counted, leading to wasted time.2. Time tracking not being updated accurately, as you actually spent 2 and a half hours on the task.Is this a better representation of your concerns?"

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "Please retain the key points of the user's input in bullet points and delete the rest of the content, ensuring that no changes are made to any key point's content."},
        {"role": "user", "content": i},
    ]
)
print(completion.choices[0].message.content)

# Delete Assistant

In [ ]:
# client.beta.assistants.delete(assistant.id)

In [ ]:
client.beta.assistants.list(
    order="desc",
    limit="20",
)